In [1]:
"""
RNN은 기존의 뉴럴넷과 다른점이 있다면, 이전의 output이 다음의 cell로 연결된다는 것.
텐서플로우를 사용하면 쉽게 구현가능
"""

'\nRNN은 기존의 뉴럴넷과 다른점이 있다면, 이전의 output이 다음의 cell로 연결된다는 것.\n텐서플로우를 사용하면 쉽게 구현가능\n'

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [3]:
# One hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]
# shape=(1,1,4) # input shape [[[1,0,0,0]]]
hidden_size=2 # hidden size를 무엇으로 정했는지에따라 출력이 결정된다.
# input data의 dimension이 뭐든 상관없이 출력의 값을 2개로 주겠다..라는 것
# -> output shape=(1,1,2)   [[[x,x]]]
# 이런건 cell을 만들때 정의하는 것임

In [4]:
"""
step 1
cell을 만들어야 한다. 뭐 어떤 형태의 cell을 만들것인가? 뭐  basic, lstm, gru
cell을 만들때 가장 중요한 것은 
cell에서 나가는 output의 size가 얼마일까(hidden_size 혹은 num_units라고 함)
"""
# cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

"""
step 2
그럼 이 cell을 만든 것을 구동을 시켜서 입력을 주고 출력을 뽑아야하는데, 
이것을 보통 드라이버라고 부를 수도 있고, 다른 형태의 함수; wrapper라고 볼수 있는데 보통
tf.nn.dynamic_rnn을 쓰게 된다.
여기에 만든 cell을 넘겨주고 우리가 원하는 input을 넘겨주게 된다.
그럼 두가지 출력을 내는데,
하나는 output, 또하나는 마지막 state의 값을 준다.
state는 우리가 잘 사용할 일이 없다
"""
# outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

'\nstep 2\n그럼 이 cell을 만든 것을 구동을 시켜서 입력을 주고 출력을 뽑아야하는데, \n이것을 보통 드라이버라고 부를 수도 있고, 다른 형태의 함수; wrapper라고 볼수 있는데 보통\ntf.nn.dynamic_rnn을 쓰게 된다.\n여기에 만든 cell을 넘겨주고 우리가 원하는 input을 넘겨주게 된다.\n그럼 두가지 출력을 내는데,\n하나는 output, 또하나는 마지막 state의 값을 준다.\nstate는 우리가 잘 사용할 일이 없다\n'

In [5]:
"""
cell생성 부분과 cell을 통해 학습하고 구동하는 부분을 나누어 줌으로써 
원하는 형태의 cell을 맘대로 바꿔줄 수가 있다.
"""

# cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)

    x_data = np.array([[[1,0,0,0]]], dtype=np.float32)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[ 0.06986494, -0.02265078]]], dtype=float32)


In [6]:
"""
shape=(1,5,4)
sequence_length: series data를 몇개를 받을 것인가에 대한 값; 한번에 cell을 몇번 펼칠 것인가
이건 입력데이터를 줄때 그 입력 어레이의 형태에 의해 알아서 결정된다.
"""
# One hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

with tf.variable_scope('two_sequences') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)
    
    # 아래와 같이 sequence size를 만들어준다. seq = 5
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    # outputs들은 각 cell의 weight들일 것임.

2 2
(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[ 0.17770417,  0.68937284],
        [ 0.06786316,  0.28095865],
        [ 0.47286847,  0.1634223 ],
        [ 0.59231973,  0.11166678],
        [-0.41298565,  0.4807488 ]]], dtype=float32)


In [7]:
# batching input
"""
batch size를 통해 여러데이터를 줄 수 있음.
"""

with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ 0.12355028,  0.14976273],
        [ 0.03199994,  0.13515949],
        [ 0.21534435,  0.19047116],
        [ 0.3102963 ,  0.22424267],
        [ 0.12143589,  0.09742174]],

       [[-0.04306879,  0.02006755],
        [-0.19016083, -0.12135446],
        [ 0.03798136, -0.03051605],
        [ 0.2019273 ,  0.04879034],
        [ 0.30317193,  0.12097759]],

       [[ 0.17746802,  0.0670521 ],
        [ 0.28810859,  0.13191102],
        [ 0.09361219,  0.11960442],
        [ 0.03508233,  0.1101391 ],
        [ 0.2188635 ,

In [8]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.02298108,  0.03827505],
        [-0.01660769,  0.0673784 ],
        [-0.01992308,  0.09115271],
        [-0.02127427,  0.12768787],
        [-0.01037671, -0.03410594]],

       [[-0.00406266,  0.03409239],
        [ 0.01195425, -0.14932822],
        [ 0.01309387, -0.03287434],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.00216312,  0.0592601 ],
        [-0.00722144,  0.1042966 ],
        [-0.01352181,  0.16157055],
        [-0.02126524,  0.15845685],
        [ 0.        ,

In [9]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim(4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size=2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                        initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.01479058,  0.08751657],
        [ 0.10128039,  0.09110713],
        [ 0.17514136, -0.01490271],
        [ 0.20155185, -0.12458292],
        [ 0.1491058 ,  0.05986462]],

       [[ 0.10309401,  0.05207301],
        [ 0.07751615,  0.17546369],
        [ 0.17291632,  0.03411548],
        [ 0.20253746, -0.09338085],
        [ 0.21093722, -0.17432097]],

       [[ 0.11367605, -0.11007044],
        [ 0.16244458, -0.18185474],
        [ 0.18534388, -0.02177148],
        [ 0.22163045,  0.02945917],
        [ 0.23751295,

In [10]:
# Create input data
batch_size=3
sequence_length=5
input_dim=3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)


In [11]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dum (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                        initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ -1.07050009e-01,   6.00064099e-02,  -8.09521377e-02,
          -1.15258425e-01,  -1.10853501e-01],
        [ -2.76925325e-01,   1.89776607e-02,  -5.93931638e-02,
           3.67132165e-02,  -3.82090837e-01],
        [ -2.37706795e-01,   2.61245295e-03,  -1.20805688e-02,
           2.57515103e-01,  -6.06447041e-01],
        [ -1.41193748e-01,   3.16908292e-04,  -1.91839423e-03,
           3.97887677e-01,  -7.47776091e-01],
        [ -7.55769834e-02,   3.76467797e-05,  -2.96965009e-04,
           4.76116806e-01,  -8.41632843e-01]],

       [[ -3.79775055e-02,   5.67152483e-06,  -4.18035634e-05,
           5.33180654e-01,  -4.20296967e-01],
        [ -2.21759137e-02,   6.10068639e-07,  -8.26958694e-06,
           5.57956219e-01,  -7.07988918e-01],
        [ -1.06543126e-02,   6.54783960e-08,  -1.25419785e-06,
           5.79231739e-01,  -8.60566199e-01],
        [ -5.16101578e-03,   7.34453609e-09,  -1.93413769e-07,
           5.98074496e-01,  -9.36282694e-01],
        [ -2.5446

In [12]:
""" Cost: sequence_loss 
그 후에 output이 얼마나 좋은가를 loss를 통해 계산을 해야함. 그래야 학습이 가능하지
나온 값의 sequence를 다 알고 있으니 각각을 뭐 cross_entropy같은 걸로 계산해서 평균을 낸다던지 해서
할 수 있긴하다. 근데 복잡해질 수 있으니
tensorflow에서 sequence_loss라는 것이 있음.


"""

# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]

# onehot으로 나타냄(softmax랑 상당히 유사)
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32) 

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

"""
logits: 우리의 예측
targets: true data
weights: 각각의 자리를 얼마나 중요하게 생각하느냐.. 그냥 모두다 1이라 생각
"""
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print('Loss: ', sequence_loss.eval())

Loss:  0.596759


In [13]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim]
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]],
                         dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]],
                         dtype=tf.float32) # prediction1보다 강하게 1로 예측

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
     "Loss2: ", sequence_loss2.eval())
"""
원하는 값이 실제에 가까워질수록 loss가 1에 비해 낮아진다.
"""

Loss1:  0.513015 Loss2:  0.371101


'\n원하는 값이 실제에 가까워질수록 loss가 1에 비해 낮아진다.\n'

In [14]:
# Create input data
batch_size=3
sequence_length=5
input_dim=3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)  # batch, sequence_length, input_dim

with tf.variable_scope('MultiRNNCell') as scope:
    # Make rnn
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] * 4, state_is_tuple=True) # 3 layer
    
    # rnn in/out
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    print('dynamic rnn: ', outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)
dynamic rnn:  Tensor("MultiRNNCell/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[  2.27656172e-04,   2.16634973e-04,   1.47411658e-04,
           1.17256393e-04,   1.59535746e-04],
        [  9.91133973e-04,   1.02063862e-03,   6.32673269e-04,
           5.82273933e-04,   6.01870124e-04],
        [  2.58713146e-03,   2.33225478e-03,   1.51518453e-03,
           1.50144601e-03,   1.55601243e-03],
        [  5.12063829e-03,   3.75815760e-03,   2.66761472e-03,
           2.85365968e-03,   3.16592283e-03],
        [  8.50854255e-03,   4.8596705

In [15]:
"""
Dynamic RNN의 장점은 확실하게 없는 데이터를 0으로 만들어줘서 loss가 잘 작동하도록 한다.
sequence_length를 주기만 하면 된다.
"""
with tf.variable_scope('dynamic_rnn') as scope:
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                        sequence_length=[5, 3, 2]) # sequence_length를 임의로 줘보자
    # length 1 for batch 1, length 2 for batch 2
    
    print('dynamic rnn: ', outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

dynamic rnn:  Tensor("dynamic_rnn/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[ -6.57164529e-02,  -8.03739876e-02,   2.22690985e-01,
           1.24483649e-02,  -1.84512585e-01],
        [ -4.68223952e-02,   2.09278211e-01,   4.46212053e-01,
           1.48645230e-03,  -2.55340040e-01],
        [ -1.97796002e-02,   2.19276577e-01,   5.38733900e-01,
           3.40516526e-05,  -1.27545893e-01],
        [ -5.53835649e-03,   1.89141750e-01,   5.62418818e-01,
           4.34651838e-07,  -3.80422734e-02],
        [ -1.04363670e-03,   1.64377004e-01,   5.55978000e-01,
           4.57295801e-09,  -1.14335306e-02]],

       [[  8.71740049e-05,   1.56013623e-01,   3.68248997e-03,
           3.52323795e-11,  -3.38438665e-03],
        [  3.38054815e-05,   1.29815146e-01,   4.62183775e-03,
           4.34483477e-13,  -1.13829586e-03],
        [  8.62929755e-06,   1.11826949e-01,   4.79643745e-03,
           4.69142284e-15,  -3.82726983e-04],
        [  0.00000000e+00,   0.00000000e+0